In [ ]:
import os
from dotenv import find_dotenv, load_dotenv
from pathlib import Path
import sys
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import SeparableConv2D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import DepthwiseConv2D

absProjectDir = Path(os.getcwd()).resolve().parents[0]
projectDir = os.path.relpath(absProjectDir,os.curdir)
load_dotenv(find_dotenv())

trainDfPath = os.path.join(projectDir, os.environ.get("REF_PROC_TRAIN_DF"))
testDfPath = os.path.join(projectDir, os.environ.get("REF_PROC_TEST_DF"))
testOrigDir = os.path.join(projectDir, os.environ.get("PROC_TEST_ORIG_DIR"))
testAugmDir = os.path.join(projectDir, os.environ.get("PROC_TEST_AUG_DIR"))
trainOrigDir = os.path.join(projectDir, os.environ.get("PROC_TRAIN_ORIG_DIR"))
trainAugmDir = os.path.join(projectDir, os.environ.get("PROC_TRAIN_AUG_DIR"))
testRootDir = os.path.commonpath([testOrigDir, testAugmDir])
trainRootDir = os.path.commonpath([trainOrigDir, trainAugmDir])

sys.path.append(os.path.join(projectDir,'src/data'))
sys.path.append(os.path.join(projectDir,'src/models'))
import imggen as imgen
from customlayer import DepthWiseConvBatchNorm
from customlayer import ConvBatchNorm
from customlayer import InceptionType
from customlayer import CustomInceptionFactory

from basemodel import BaseModel
from basemodel import DepthWiseConvModel1
from basemodel import ConvModel1
from basemodel import InterceptionModelV1
from basemodel import InterceptionModelV2
from basemodel import StemDecorator
from basemodel import StemDecoratorConv
from basemodel import StemDecoratorDepthWiseConv

In [ ]:
trgen = imgen.Generator(trainDfPath, trainRootDir, (224,224))
trgen.SetDataFrameForGeneration([[2],[3]],[[256],[256]])
tsgen = imgen.Generator(testDfPath,testRootDir, (224,224))
tsgen.SetDataFrameForGeneration([[2],[3]], [[32],[32]])
train_set = trgen.GetGenerator(bs = 16)
test_set = tsgen.GetGenerator(bs = 16)

In [ ]:
intercept = StemDecoratorConv(conv_layer_cnt = 2, intercept_module = InterceptionModelV2())
model = intercept.Create()

In [ ]:
hist = model.fit(train_set,
                 steps_per_epoch = 128 / 8,
                 epochs = 1, validation_data = test_set,validation_steps = 2)

In [ ]:
m1 = ConvModel1()
model1 = m1.Create()
model1.summary()

In [ ]:
m2 = DepthWiseConvModel1()
model2 = m2.Create()
model2.summary()